# Loading data with TimeBasedCesnetDataset

### Import

In [1]:
from tqdm import tqdm
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TimeFormat, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import TimeBasedConfig # Time based dataset MUST use TimeBasedConfig

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
time_based_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.INSTITUTION_SUBNETS, aggregation=AgreggationType.AGG_1_HOUR, dataset_type=DatasetType.TIME_BASED, display_details=True)

[2025-08-26 20:08:31,803][wrapper_dataset][INFO] - Dataset is time-based. Use cesnet_tszoo.configs.TimeBasedConfig



Dataset details:

    AgreggationType.AGG_1_HOUR
        Time indices: range(0, 6717)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 0, tzinfo=datetime.timezone.utc))

    SourceType.INSTITUTION_SUBNETS
        Time series indices: [0 1 2 3 4 ... 543 544 545 546 547], Length=548; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0, 'sum_n_dest_asn': 0, 'avg_n_dest_asn': 0, 'std_n_dest_asn': 0, 'sum_n_dest_ports': 0, 'avg_n_dest_ports': 0, 'std_n_dest_ports': 0, 'sum_n_dest_ip': 0, 'avg_n_dest_ip': 0, 'std_n_dest_ip': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Loading data with DataLoader

- Load data using Pytorch Dataloader.
- Last batch is never dropped (unless sliding_window is used).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_dataloader` with parameter `workers`.
- Batch size affect how many times for every time series will be in one batch (differs when sliding window is used).
- Batch consists of: (only when sliding window is not used).
    - When `time_format` is not TimeFormat.DATETIME, then batch is one Numpy array of shape `(ts_ids, batch_size, features_to_take + used ids)`.
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: (Numpy array of shape `(ts_ids, batch_size, features_to_take + used ids (without time))`, Numpy array of shape `(batch_size)`)

In [4]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:08:31,808][time_config][INFO] - Quick validation succeeded.
[2025-08-26 20:08:31,828][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:08:31,833][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 483.89it/s]
[2025-08-26 20:08:31,952][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [270 125 255 116  54 ... 401 163 468 278 350], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    

You can also change set batch sizes later with `update_dataset_config_and_initialize` or `set_batch_sizes`.

In [5]:
time_based_dataset.update_dataset_config_and_initialize(train_batch_size=33, val_batch_size=65, test_batch_size="config", all_batch_size="config")
# Or
time_based_dataset.set_batch_sizes(train_batch_size=33, val_batch_size=65, test_batch_size="config", all_batch_size="config")

[2025-08-26 20:08:31,957][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:08:31,958][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:08:31,959][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:08:31,960][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:08:31,960][cesnet_dataset][INFO] - Batch sizes has been changed successfuly.


You can also change set workers later with `update_dataset_config_and_initialize` or `set_workers`.

In [6]:
time_based_dataset.update_dataset_config_and_initialize(train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
# Or
time_based_dataset.set_workers(train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)

[2025-08-26 20:08:31,965][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:08:31,966][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:08:31,967][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:08:31,967][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:08:31,968][cesnet_dataset][INFO] - Workers has been changed successfuly.


#### Train set

- Affected by `train_batch_size`.
- Affected by `train_workers`.

In [7]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 20:08:31,975][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 220.51it/s]


(54, 32, 20)

#### Val set

- Affected by `val_batch_size`.
- Affected by `val_workers`.

In [8]:
dataloader = time_based_dataset.get_val_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 20:08:32,464][cesnet_dataset][INFO] - Created new cached val_dataloader.
100%|██████████| 32/32 [00:00<00:00, 140.69it/s]


(54, 64, 20)

#### Test set

- Affected by `test_batch_size`.
- Affected by `test_workers`.

In [9]:
dataloader = time_based_dataset.get_test_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 20:08:32,702][cesnet_dataset][INFO] - Created new cached test_dataloader.
100%|██████████| 11/11 [00:00<00:00, 116.91it/s]


(54, 128, 20)

#### All set

- Affected by `all_batch_size`.
- Affected by `all_workers`.

In [10]:
dataloader = time_based_dataset.get_all_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 20:08:32,808][cesnet_dataset][INFO] - Created new cached all_dataloader.
100%|██████████| 53/53 [00:00<00:00, 186.65it/s]


(54, 128, 20)

#### Using time_format=TimeFormat.DATETIME

In [11]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:08:33,098][time_config][INFO] - Quick validation succeeded.
[2025-08-26 20:08:33,122][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:08:33,126][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 807.09it/s]
[2025-08-26 20:08:33,195][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [262 118 116 235 283 ... 298 392 458 245 218], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
   

In [12]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0][0].shape) # data without time
display(batches[0][1].shape) # time

[2025-08-26 20:08:33,205][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 237.56it/s]


(54, 32, 19)

(32,)

#### Specifying which time series to load

- Every `get_*_dataloader` has parameter `ts_id`.
    - When `ts_id` is None, then it returns as previous examples.
    - When `ts_id` is not None, then it returns only one time series of that specified id.

In [13]:
config = TimeBasedConfig(ts_ids=[177, 176, 319, 267], train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:08:33,657][time_config][INFO] - Quick validation succeeded.
[2025-08-26 20:08:33,676][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:08:33,680][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 4/4 [00:00<00:00, 533.36it/s]
[2025-08-26 20:08:33,690][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [177 176 319 267], Length=4
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Slidin

In [14]:
dataloader = time_based_dataset.get_train_dataloader(ts_id=177 , workers="config",)

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 20:08:33,697][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 2034.67it/s]


(1, 32, 20)

#### Sliding window

- Both `sliding_window_size` and `sliding_window_prediction_size` must be set if you want to use sliding window.
- Batch sizes are used for background caching.
- Batch consists of:
    - When `time_format` is not TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(ts_ids, sliding_window_size, features_to_take + used ids)`, <br>
        Numpy array of shape `(ts_ids, sliding_window_prediction_size, features_to_take + used ids)` <br>
    ).
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(ts_ids, sliding_window_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(ts_ids, sliding_window_prediction_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(sliding_window_size)` of time, <br>
        Numpy array of shape `(sliding_window_prediction_size)` of time, <br>
    ).
- You can modify sliding window step size with `sliding_window_step`
- You can use `set_shared_size` to set how many times time periods should share.
    - `val_time_period` takes from `train_time_period`
    - `test_time_period` takes from `val_time_period` or `train_time_period`

In [15]:
config = TimeBasedConfig(ts_ids=54, train_time_period=range(0, 1000), val_time_period=range(1000, 1500), test_time_period=range(1500, 2000), features_to_take=["n_flows"], time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=0.05)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:08:33,757][time_config][INFO] - Quick validation succeeded.
[2025-08-26 20:08:33,775][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:08:33,778][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 1945.66it/s]
[2025-08-26 20:08:33,808][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [412 417 438 107 387 ... 528 141 502 428  78], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(665, 1500)
        Test time periods: range(1165, 2000)
        All time periods: range(0, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
        Set shared size: 335
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Anomaly handler
        Anomaly handler type: None        
    Batch sizes
        Train batch size: 32
        Val batch size: 64

In [16]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction))

[2025-08-26 20:08:33,816][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 5088.85it/s]


You can also change sliding window parameters later with `update_dataset_config_and_initialize` or `set_sliding_window`.

In [17]:
time_based_dataset.update_dataset_config_and_initialize(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)
# Or
time_based_dataset.set_sliding_window(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)

[2025-08-26 20:08:33,917][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:08:33,918][cesnet_dataset][INFO] - Destroyed cached train_dataloader.
[2025-08-26 20:08:33,918][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:08:33,919][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:08:33,919][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:08:33,919][cesnet_dataset][INFO] - Sliding window values has been changed successfuly.


##### Using time_format=TimeFormat.DATETIME

In [18]:
config = TimeBasedConfig(ts_ids=54, train_time_period=range(0, 1000), val_time_period=range(978, 1500), test_time_period=range(1478, 2000), features_to_take=["n_flows"], time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=100)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:08:33,924][time_config][INFO] - Quick validation succeeded.
[2025-08-26 20:08:33,944][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:08:33,947][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 2255.18it/s]
[2025-08-26 20:08:33,973][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [280  48 498 246  75 ... 284 328  49 144 120], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(900, 1500)
        Test time periods: range(1400, 2000)
        All time periods: range(0, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
        Set shared size: 100
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Anomaly handler
        Anomaly handler type: None        
    Batch sizes
        Train batch size: 32
        Val batch size: 6

In [19]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times))

[2025-08-26 20:08:33,980][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 5252.28it/s]


### Loading data as Dataframe

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `ts_ids` with sets specified time period.
- Data is returned as Pandas Dataframe.
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_df` with parameter `workers`.

In [20]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:08:34,080][time_config][INFO] - Quick validation succeeded.
[2025-08-26 20:08:34,098][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:08:34,102][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 654.12it/s]
[2025-08-26 20:08:34,186][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [300 271 128  12 104 ... 275 522  49  74 236], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    

#### Train set

- Affected by `train_workers`.

In [21]:
df = time_based_dataset.get_train_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_train_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,300.0,0.0,196.0,266.0,11555.0,114.0,5.18,2.02,119.0,5.41,2.09,171.0,7.77,4.83,0.750000,0.729980,0.429932,0.409912,2.03,224.720001
1,300.0,1.0,193.0,285.0,12038.0,135.0,5.87,1.74,143.0,6.22,1.93,184.0,8.00,3.55,0.790039,0.779785,0.419922,0.399902,1.30,205.669998
2,300.0,2.0,175.0,273.0,12933.0,106.0,4.82,1.68,111.0,5.05,1.89,166.0,7.55,4.31,0.689941,0.680176,0.399902,0.389893,1.22,211.059998
3,300.0,3.0,132.0,198.0,7891.0,87.0,3.78,1.93,100.0,4.35,2.21,124.0,5.39,2.89,0.770020,0.770020,0.489990,0.479980,1.66,207.869995
4,300.0,4.0,132.0,194.0,7771.0,87.0,3.62,1.35,94.0,3.92,1.50,131.0,5.46,3.62,0.720215,0.729980,0.449951,0.439941,2.14,215.449997
5,300.0,5.0,94.0,131.0,5908.0,68.0,3.09,1.23,71.0,3.23,1.31,89.0,4.05,1.94,0.779785,0.750000,0.449951,0.439941,0.29,215.320007
6,300.0,6.0,78.0,118.0,5225.0,54.0,2.70,1.13,59.0,2.95,1.28,72.0,3.60,1.76,0.779785,0.759766,0.479980,0.449951,6.22,211.610001
7,300.0,7.0,134.0,196.0,7799.0,85.0,3.70,1.66,94.0,4.09,2.23,126.0,5.48,3.50,0.759766,0.759766,0.439941,0.429932,1.13,225.520004
8,300.0,8.0,96.0,151.0,7186.0,69.0,3.83,1.42,68.0,3.78,1.40,93.0,5.17,2.28,0.669922,0.629883,0.459961,0.419922,0.65,219.350006
9,300.0,9.0,86.0,152.0,7291.0,68.0,3.40,1.50,72.0,3.60,1.64,84.0,4.20,2.50,0.799805,0.779785,0.469971,0.459961,1.28,227.410004


In [22]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets  n_bytes  \
 0                     300.0      0.0    196.0      266.0  11555.0   
 1                     300.0      1.0    193.0      285.0  12038.0   
 2                     300.0      2.0    175.0      273.0  12933.0   
 3                     300.0      3.0    132.0      198.0   7891.0   
 4                     300.0      4.0    132.0      194.0   7771.0   
 ...                     ...      ...      ...        ...      ...   
 3354                  300.0   3354.0     77.0      106.0   5584.0   
 3355                  300.0   3355.0     93.0      120.0   4760.0   
 3356                  300.0   3356.0     57.0       74.0   2868.0   
 3357                  300.0   3357.0     77.0       99.0   4753.0   
 3358                  300.0   3358.0     77.0      105.0   4221.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              114.0            5.18            2.02             119.0   
 1      

#### Val set

- Affected by `val_workers`.

In [23]:
df = time_based_dataset.get_val_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_val_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,300.0,3359.0,130.0,175.0,7127.0,67.0,3.72,1.49,71.0,3.94,1.59,119.0,6.61,4.80,0.629883,0.629883,0.489990,0.469971,2.70,222.830002
1,300.0,3360.0,141.0,211.0,8915.0,66.0,3.30,1.81,64.0,3.20,1.67,135.0,6.75,6.12,0.669922,0.640137,0.569824,0.540039,3.06,213.149994
2,300.0,3361.0,155.0,250.0,9563.0,71.0,3.55,1.88,70.0,3.50,1.67,147.0,7.35,5.55,0.509766,0.520020,0.449951,0.439941,6.73,219.270004
3,300.0,3362.0,89.0,121.0,4980.0,61.0,3.21,1.03,62.0,3.26,1.10,82.0,4.32,1.67,0.680176,0.689941,0.520020,0.509766,2.59,223.070007
4,300.0,3363.0,116.0,153.0,6079.0,59.0,3.11,1.56,61.0,3.21,1.72,109.0,5.74,4.90,0.680176,0.680176,0.509766,0.489990,3.28,220.169998
5,300.0,3364.0,77.0,105.0,4233.0,45.0,2.50,1.15,43.0,2.39,1.09,77.0,4.28,2.95,0.569824,0.589844,0.589844,0.580078,1.91,240.850006
6,300.0,3365.0,101.0,152.0,5876.0,47.0,2.61,1.46,43.0,2.39,1.29,96.0,5.33,4.79,0.589844,0.600098,0.500000,0.489990,4.02,222.520004
7,300.0,3366.0,93.0,145.0,5959.0,48.0,2.67,0.84,50.0,2.78,1.11,89.0,4.94,2.71,0.569824,0.580078,0.560059,0.520020,4.48,232.000000
8,300.0,3367.0,104.0,167.0,6724.0,57.0,2.71,1.01,50.0,2.38,0.92,101.0,4.81,3.70,0.529785,0.520020,0.529785,0.500000,4.00,229.520004
9,300.0,3368.0,116.0,164.0,6269.0,57.0,3.00,1.60,48.0,2.53,1.26,112.0,5.89,5.05,0.489990,0.489990,0.549805,0.540039,6.74,225.720001


In [24]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets  n_bytes  \
 0                     300.0   3359.0    130.0      175.0   7127.0   
 1                     300.0   3360.0    141.0      211.0   8915.0   
 2                     300.0   3361.0    155.0      250.0   9563.0   
 3                     300.0   3362.0     89.0      121.0   4980.0   
 4                     300.0   3363.0    116.0      153.0   6079.0   
 ...                     ...      ...      ...        ...      ...   
 2010                  300.0   5369.0     50.0       69.0   2980.0   
 2011                  300.0   5370.0     73.0      100.0   4738.0   
 2012                  300.0   5371.0     60.0       86.0   3795.0   
 2013                  300.0   5372.0     56.0       76.0   3698.0   
 2014                  300.0   5373.0     60.0       90.0   3735.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0               67.0            3.72            1.49              71.0   
 1      

#### Test set

- Affected by `test_workers`.

In [25]:
df = time_based_dataset.get_test_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_test_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,300.0,5374.0,72.0,85.0,3787.0,43.0,2.69,1.70,51.0,3.19,1.83,68.0,4.25,3.19,0.700195,0.700195,0.500000,0.469971,2.61,221.020004
1,300.0,5375.0,74.0,95.0,4650.0,50.0,3.12,1.09,54.0,3.38,1.50,71.0,4.44,2.66,0.709961,0.680176,0.469971,0.419922,2.86,205.460007
2,300.0,5376.0,70.0,93.0,4230.0,38.0,2.24,1.15,45.0,2.65,1.32,65.0,3.82,2.10,0.709961,0.680176,0.479980,0.439941,3.73,198.190002
3,300.0,5377.0,69.0,94.0,3911.0,44.0,3.14,1.56,49.0,3.50,1.87,61.0,4.36,3.23,0.740234,0.750000,0.509766,0.479980,2.95,220.380005
4,300.0,5378.0,30.0,43.0,1788.0,20.0,1.33,0.49,23.0,1.53,0.64,29.0,1.93,1.53,0.870117,0.870117,0.620117,0.589844,4.92,193.580002
5,300.0,5379.0,37.0,53.0,2685.0,28.0,2.55,1.86,34.0,3.09,2.02,36.0,3.27,2.10,0.859863,0.819824,0.520020,0.479980,1.85,196.130005
6,300.0,5380.0,35.0,50.0,2116.0,30.0,1.88,1.15,32.0,2.00,1.15,34.0,2.12,1.09,0.859863,0.859863,0.540039,0.500000,1.67,198.259995
7,300.0,5381.0,34.0,47.0,2020.0,28.0,2.00,1.24,27.0,1.93,1.21,33.0,2.36,1.45,0.620117,0.620117,0.600098,0.580078,3.86,227.960007
8,300.0,5382.0,48.0,66.0,2688.0,30.0,2.50,1.17,34.0,2.83,1.64,45.0,3.75,2.30,0.750000,0.759766,0.629883,0.609863,3.63,228.199997
9,300.0,5383.0,55.0,73.0,3210.0,40.0,2.86,1.35,46.0,3.29,1.59,53.0,3.79,1.67,0.750000,0.750000,0.520020,0.489990,0.08,203.149994


In [26]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets  n_bytes  \
 0                     300.0   5374.0     72.0       85.0   3787.0   
 1                     300.0   5375.0     74.0       95.0   4650.0   
 2                     300.0   5376.0     70.0       93.0   4230.0   
 3                     300.0   5377.0     69.0       94.0   3911.0   
 4                     300.0   5378.0     30.0       43.0   1788.0   
 ...                     ...      ...      ...        ...      ...   
 1338                  300.0   6712.0     86.0      112.0   4823.0   
 1339                  300.0   6713.0     67.0       90.0   4157.0   
 1340                  300.0   6714.0     96.0      141.0   6221.0   
 1341                  300.0   6715.0     96.0      134.0   5978.0   
 1342                  300.0   6716.0     68.0       91.0   3718.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0               43.0            2.69            1.70              51.0   
 1      

In [27]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets  n_bytes  \
 0                     300.0   5374.0     72.0       85.0   3787.0   
 1                     300.0   5375.0     74.0       95.0   4650.0   
 2                     300.0   5376.0     70.0       93.0   4230.0   
 3                     300.0   5377.0     69.0       94.0   3911.0   
 4                     300.0   5378.0     30.0       43.0   1788.0   
 ...                     ...      ...      ...        ...      ...   
 1338                  300.0   6712.0     86.0      112.0   4823.0   
 1339                  300.0   6713.0     67.0       90.0   4157.0   
 1340                  300.0   6714.0     96.0      141.0   6221.0   
 1341                  300.0   6715.0     96.0      134.0   5978.0   
 1342                  300.0   6716.0     68.0       91.0   3718.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0               43.0            2.69            1.70              51.0   
 1      

#### All set

- Affected by `all_workers`.

In [28]:
df = time_based_dataset.get_all_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_all_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,300.0,0.0,196.0,266.0,11555.0,114.0,5.18,2.02,119.0,5.41,2.09,171.0,7.77,4.83,0.750000,0.729980,0.429932,0.409912,2.03,224.720001
1,300.0,1.0,193.0,285.0,12038.0,135.0,5.87,1.74,143.0,6.22,1.93,184.0,8.00,3.55,0.790039,0.779785,0.419922,0.399902,1.30,205.669998
2,300.0,2.0,175.0,273.0,12933.0,106.0,4.82,1.68,111.0,5.05,1.89,166.0,7.55,4.31,0.689941,0.680176,0.399902,0.389893,1.22,211.059998
3,300.0,3.0,132.0,198.0,7891.0,87.0,3.78,1.93,100.0,4.35,2.21,124.0,5.39,2.89,0.770020,0.770020,0.489990,0.479980,1.66,207.869995
4,300.0,4.0,132.0,194.0,7771.0,87.0,3.62,1.35,94.0,3.92,1.50,131.0,5.46,3.62,0.720215,0.729980,0.449951,0.439941,2.14,215.449997
5,300.0,5.0,94.0,131.0,5908.0,68.0,3.09,1.23,71.0,3.23,1.31,89.0,4.05,1.94,0.779785,0.750000,0.449951,0.439941,0.29,215.320007
6,300.0,6.0,78.0,118.0,5225.0,54.0,2.70,1.13,59.0,2.95,1.28,72.0,3.60,1.76,0.779785,0.759766,0.479980,0.449951,6.22,211.610001
7,300.0,7.0,134.0,196.0,7799.0,85.0,3.70,1.66,94.0,4.09,2.23,126.0,5.48,3.50,0.759766,0.759766,0.439941,0.429932,1.13,225.520004
8,300.0,8.0,96.0,151.0,7186.0,69.0,3.83,1.42,68.0,3.78,1.40,93.0,5.17,2.28,0.669922,0.629883,0.459961,0.419922,0.65,219.350006
9,300.0,9.0,86.0,152.0,7291.0,68.0,3.40,1.50,72.0,3.60,1.64,84.0,4.20,2.50,0.799805,0.779785,0.469971,0.459961,1.28,227.410004


In [29]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets  n_bytes  \
 0                     300.0      0.0    196.0      266.0  11555.0   
 1                     300.0      1.0    193.0      285.0  12038.0   
 2                     300.0      2.0    175.0      273.0  12933.0   
 3                     300.0      3.0    132.0      198.0   7891.0   
 4                     300.0      4.0    132.0      194.0   7771.0   
 ...                     ...      ...      ...        ...      ...   
 6712                  300.0   6712.0     86.0      112.0   4823.0   
 6713                  300.0   6713.0     67.0       90.0   4157.0   
 6714                  300.0   6714.0     96.0      141.0   6221.0   
 6715                  300.0   6715.0     96.0      134.0   5978.0   
 6716                  300.0   6716.0     68.0       91.0   3718.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              114.0            5.18            2.02             119.0   
 1      

### Loading data as singular Numpy array 

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `ts_ids` with sets specified time period.
- Data is returned as one Numpy array.
- Follows similar rules to Dataloader batches, regarding shape (excluding sliding window parameters).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_numpy` with parameter `workers`.

In [30]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:08:35,955][time_config][INFO] - Quick validation succeeded.
[2025-08-26 20:08:35,974][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:08:35,977][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 625.24it/s]
[2025-08-26 20:08:36,066][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [371   5 455 339 535 ... 157  98 504 114 393], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    

#### Train set

- Affected by `train_workers`.

In [31]:
numpy_array = time_based_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)

(54, 3359, 20)

#### Val set

- Affected by `val_workers`.

In [32]:
numpy_array = time_based_dataset.get_val_numpy(workers="config")

display(numpy_array.shape)

(54, 2015, 20)

#### Test set

- Affected by `test_workers`.

In [33]:
numpy_array = time_based_dataset.get_test_numpy(workers="config")

display(numpy_array.shape)

(54, 1343, 20)

#### All set

- Affected by `all_workers`.

In [34]:
numpy_array = time_based_dataset.get_all_numpy(workers="config")

display(numpy_array.shape)

(54, 6717, 20)

#### Using time_format=TimeFormat.DATETIME

In [35]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:08:36,358][time_config][INFO] - Quick validation succeeded.
[2025-08-26 20:08:36,381][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:08:36,385][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 1039.21it/s]
[2025-08-26 20:08:36,440][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [345  58  55 191 123 ...  72 143  51 542 442], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
   

In [36]:
numpy_array, times = time_based_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)
display(times)

(54, 3359, 19)

array([datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 1, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 2, 0, tzinfo=datetime.timezone.utc),
       ...,
       datetime.datetime(2024, 2, 25, 20, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 21, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 22, 0, tzinfo=datetime.timezone.utc)],
      shape=(3359,), dtype=object)